In [5]:
import numpy as np
import matplotlib.pyplot as plt
import sjoert.stellar
import pandas as pd
import os
from tqdm import tqdm
from astropy import coordinates as coord
from scipy.optimize import curve_fit
import forced_phot as fp #import flux_jy, chi2, gaussian, chi2_peak_finder
from scipy.constants import h,c,k
import json

sjoertpath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares'

The below shouldn't be in the class but should be the loop in which the class is called.

In [34]:
test_ztf = 'ZTF18abjjkeo'
cwd = os.getcwd()
dir = cwd
while not dir.endswith('First Research Project\\Data'):
    cwd = os.path.split(cwd)[0]
    dir = cwd + "\\Data"
banlist = set(['DOWNLOADED','EMPTY','dump'])
iterlist = sorted(set(os.listdir(dir)) - banlist)

for folder in iterlist:
    folder_dir = os.path.join(dir,folder)
    for ztf_name in os.listdir(folder_dir):
        # call the class
        pass


ZTF20aagkqky


In [49]:
def BB(nu,T):
    #Blackbody spectrum for a certain frequency given in Hz, not an array of values
    factor = 2*h*np.power(nu,3)/(c**2)
    exponent = (h*nu)/(k*T)
    return factor /(np.exp(exponent)-1)

def BB_ratio(T,v1,v0):
    #very simple, just for brevity
    return BB(v1,T)/BB(v0,T)

#With BB temperature correction, used for fitting to ZTF_g and ZTF_r data
def gauss_exp_fit(t,*p):
    Fp = 10**p[0]
    peak_position = p[1]
    sigma_rise = 10**p[2]
    tau_dec = 10**p[3]
    F0 = p[4]
    T = 10**p[5]

    trel = t - peak_position
    gaussian = lambda t: Fp * np.exp(-(np.square(t-peak_position)/(2*sigma_rise**2))) + F0
    exp_decay = lambda t: Fp * np.exp(-(t-peak_position)/tau_dec) + F0

    function = np.piecewise(t,[trel <= 0,trel>0],[gaussian, exp_decay])
    return function * BB_ratio(T)

#without the BB correction, used for plotting. 
def gauss_exp(t,*p):
    Fp = 10**p[0]
    peak_position = p[1]
    sigma_rise = 10**p[2]
    tau_dec = 10**p[3]
    F0 = p[4]
    T = 10**p[5]

    trel = t - peak_position
    gaussian = lambda t: Fp * np.exp(-(np.square(t-peak_position)/(2*sigma_rise**2))) + F0
    exp_decay = lambda t: Fp * np.exp(-(t-peak_position)/tau_dec) + F0

    # trel2 = t - t[np.argmin(gaussian(t)-Fp)]
    # print(trel2)
    function = np.piecewise(t,[trel <= 0,trel>0],[gaussian, exp_decay])
    return function 


In [52]:
class ZTF_forced_phot:
    def __init__(self,ztf_name): #ztf_name may be simply the name or the path leading to where the relevant data is stored if available.
        if "\\" in ztf_name: #ergo a directory was given instead of a name
            ztf_dir = ztf_name #make self(?)
            ztf_name = os.path.split(ztf_dir)[-1]
        else:
            cwd = os.getcwd()
            dir = cwd
            while not dir.endswith('First Research Project\\Data'):
                cwd = os.path.split(cwd)[0]
                dir = cwd + "\\Data"
            banlist = set(['DOWNLOADED','EMPTY','dump'])
            iterlist = sorted(set(os.listdir(dir)) - banlist)

            for folder in iterlist:
                folder_dir = os.path.join(dir,folder)
                if 'Sjoert' in folder_dir: #REMOVE THIS LATER, ONLY FOR TESTING
                    if ztf_name in os.listdir(folder_dir):
                        ztf_dir = os.path.join(folder_dir,ztf_name) #make self (?)
                        break

        clean_data = pd.read_json(os.path.join(ztf_dir,f"{ztf_name}_clean_data.json")) #make self
        clean_data.sort_values('time',inplace=True) #sort the data by time for plotting reasons later on
        no_i_mask = clean_data['filter'] != 'ZTF_i' #create the ZTf_i mask
        flux,err = fp.flux_jy(clean_data) #convert the flux and its errors to uJY
        err = np.clip(err,0.01*flux,np.inf) # clip the errors to be at least 1% of the flux
        clean_data['flux'] = flux.values
        clean_data['flux_unc'] = err.values

        time_zeropoint = np.min(clean_data['time'][no_i_mask]) #find the zeropoint for time to be used here
        time_mjd = clean_data['time'] - time_zeropoint
        clean_data['time'] = time_mjd.values #transform time column

        with open(os.path.join(ztf_dir,f"{ztf_name}_clean_log.json")) as f:
            logfile = json.load(f)

        columns = ['sindex', 'field', 'ccdid', 'qid', 'filter', 'pid', 'infobitssci', 'sciinpseeing', 'scibckgnd', 'scisigpix', 'zpmaginpsci', 'zpmaginpsciunc', 'zpmaginpscirms', 'clrcoeff', 'clrcoeffunc', 'ncalmatches', 'exptime', 'adpctdif1', 'adpctdif2', 'diffmaglim', 'zpdiff', 'programid', 'jd', 'rfid', 'forcediffimflux', 'forcediffimfluxunc', 'forcediffimsnr', 'forcediffimchisq', 'forcediffimfluxap', 'forcediffimfluxuncap', 'forcediffimsnrap', 'aperturecorr', 'dnearestrefsrc', 'nearestrefmag', 'nearestrefmagunc', 'nearestrefchi', 'nearestrefsharp', 'refjdstart', 'refjdend', 'procstatus']
        dtypes = [(columns[x],float) for x in range(len(columns))]
        dtypes[4] = ('filter',r'U8')
        for file in os.listdir(ztf_dir):
            if 'batchfp' in file:
                batchrq_string = file
        unclean_data = pd.DataFrame(np.genfromtxt(os.path.join(ztf_dir,batchrq_string),skip_header=53,dtype=dtypes))
        unclean_data = unclean_data[unclean_data['forcediffimfluxunc'] > 0] #these need to be removed or plotting doesn't work, usually not a lot of datapoints.


        #Source: http://svo2.cab.inta-csic.es/theory/fps/index.php?mode=browse&gname=Palomar&gname2=ZTF&asttype=
        g_center = c/ (4746.48 * 1e-10)
        r_center = c / (6366.38 * 1e-10)

        chi2_results, peak_ind, peak_position_guess, peak_guess = fp.chi2_peak_finder(flux.values,err.values,clean_data['time'],time_zeropoint)
        time_mask = (clean_data['time'] > (peak_position_guess - 365)) & (clean_data['time'] < (peak_position_guess+365*2)) #these are the times we will be fitting on
        time_mask *= no_i_mask # also filter out ZTF_i measurements
        
        #values for fitting
        flux_fit,err_fit,time_fit,filters_fit = flux[time_mask],err[time_mask],clean_data['time'][time_mask],clean_data['filter'][time_mask].to_numpy()

        #frequency array and "central" frequency for blackbody ratio correction
        v1 = filters_fit.copy()
        v1[v1 == 'ZTF_g'] = g_center
        v1[v1 == 'ZTF_r'] = r_center
        v1 = v1.astype(np.float64)
        v0 = np.average([g_center,r_center],weights=[np.sum(filters_fit=='ZTF_g'),np.sum(filters_fit=='ZTF_r')])

        #Initial guesses and boundings for fitting in order: Fp, peak_pos, sigma, tau_dec, F0, T
        guesses = [np.log10(np.max(flux_fit)),peak_position_guess,1,2.5,1,4]
        boundings = ([1,peak_position_guess-100,0,0,0,3],[np.log10(np.max(flux_fit*2)),peak_position_guess+100,4,4,np.median(flux_fit),5]) 



        #initializing all the variables that need to be used later on with self.
        self.ztf_name = ztf_name
        self.ztf_dir = ztf_dir

        self.no_i_mask = no_i_mask
        self.time_zeropoint = time_zeropoint
        self.time_mask = time_mask

        self.clean_data = clean_data
        self.flux, self.err, self.time = self.clean_data[['flux','flux_unc','time']].T.to_numpy(dtype=np.float64)
        self.flux_fit,self.err_fit,self.time_fit,self.filters_fit = flux_fit,err_fit,time_fit,filters_fit
        self.clean_filtermasks = [(self.clean_data['filter'] == 'ZTF_g'), (self.clean_data['filter'] == 'ZTF_r'), (self.clean_data['filter'] == 'ZTF_i')]
        self.logfile = logfile
        self.unclean_data = unclean_data
        self.g_center = g_center
        self.r_center = r_center
        self.v0, self.v1 = v0, v1
        self.guesses, self.boundings= guesses, boundings
        self.chi2_results, self.peak_ind, self.peak_position_guess, self.peak_guess = chi2_results, peak_ind, peak_position_guess, peak_guess


    def plot_clean_unclean_data(self):
        clean_flux, clean_err, clean_time = self.flux,self.err,self.time
        raw_flux, raw_err, raw_time = self.unclean_data[['forcediffimflux','forcediffimfluxunc','jd']].T.to_numpy(dtype=np.float64)
        raw_flux = sjoert.stellar.mag2flux(self.unclean_data['zpdiff']) * 1e6 * raw_flux
        raw_err = sjoert.stellar.mag2flux(self.unclean_data['zpdiff']) * 1e6 * raw_err
        raw_time -= self.time_zeropoint

        raw_filtersmasks = [(self.unclean_data['filter'] == 'ZTF_g'), (self.unclean_data['filter'] == 'ZTF_r'), (self.unclean_data['filter'] == 'ZTF_i')]
        colors = ['green','red','brown']
        names = ['ZTF_g','ZTF_r','ZTF_i']

        fig,axes = plt.subplots(nrows=3,sharex=True,figsize=(6,8))
        plt.suptitle(self.ztf_name,fontsize=14)
        axes[0].xaxis.set_tick_params(which='both', labelbottom=True)
        axes[1].xaxis.set_tick_params(which='both', labelbottom=True)
        for i,ax in enumerate(axes):
            ax.set_title(names[i])
            ax.errorbar(raw_time[raw_filtersmasks[i]],raw_flux[raw_filtersmasks[i]],raw_err[raw_filtersmasks[i]],fmt=',',alpha=0.5,c='gray',label='Raw data')
            ax.errorbar(clean_time[self.clean_filtermasks[i]],clean_flux[self.clean_filtermasks[i]],clean_err[self.clean_filtermasks[i]],fmt=',',c=colors[i],label='Cleaned data')
            ax.legend(loc='lower right')
            ax.set_ylabel(r'Flux [$\mu$Jy]',fontsize=12)
        
        plt.xlabel(f'Time [mjd] w.r.t. JD {self.time_zeropoint}',fontsize=12)
        fig.tight_layout()
        plt.show()
    
    def BB(self,nu,T):
        #Blackbody spectrum for a certain frequency given in Hz, not an array of values
        factor = 2*h*np.power(nu,3)/(c**2)
        exponent = (h*nu)/(k*T)
        return factor /(np.exp(exponent)-1)

    def BB_ratio(self,T):
        #very simple, just for brevity
        return self.BB(self.v1,T)/self.BB(self.v0,T)
    
    #With BB temperature correction, used for fitting to ZTF_g and ZTF_r data
    def gauss_exp_fit(self,t,*p):
        Fp = 10**p[0]
        peak_position = p[1]
        sigma_rise = 10**p[2]
        tau_dec = 10**p[3]
        F0 = p[4]
        T = 10**p[5]

        trel = t - peak_position
        gaussian = lambda t: Fp * np.exp(-(np.square(t-peak_position)/(2*sigma_rise**2))) + F0
        exp_decay = lambda t: Fp * np.exp(-(t-peak_position)/tau_dec) + F0

        function = np.piecewise(t,[trel <= 0,trel>0],[gaussian, exp_decay])
        return function * self.BB_ratio(T)

    #without the BB correction, used for plotting. 
    def gauss_exp(t,*p):
        Fp = 10**p[0]
        peak_position = p[1]
        sigma_rise = 10**p[2]
        tau_dec = 10**p[3]
        F0 = p[4]
        # T = 10**p[5]

        trel = t - peak_position
        gaussian = lambda t: Fp * np.exp(-(np.square(t-peak_position)/(2*sigma_rise**2))) + F0
        exp_decay = lambda t: Fp * np.exp(-(t-peak_position)/tau_dec) + F0

        # trel2 = t - t[np.argmin(gaussian(t)-Fp)]
        # print(trel2)
        function = np.piecewise(t,[trel <= 0,trel>0],[gaussian, exp_decay])
        return function 


    def fit(self,one_plot=True,subplot=False):
        
        greenmask = self.clean_filtermasks
        popt,pcov,info_dict,_,_ = curve_fit(self.gauss_exp_fit,self.time_fit,self.flux_fit,p0=self.guesses,bounds=self.boundings,sigma=self.err_fit,
                                        full_output=True) 

        names = ["log10(Fp)","peak_position","log10(sigma_rise)","log10(tau_dec)","F0","log10(T)"]
        display = [f"{names[i]} = {opt}" for i,opt in enumerate(popt)]
        print("Popt:",display,'\nErrors',np.diagonal(pcov),'\nRelative error: ',np.diagonal(pcov)/popt)

        # time_fit -= popt[1]
        # time -= popt[1]

        moretimes = np.linspace(min(self.time_fit),max(self.time_fit),2000)
        fit = self.gauss_exp(moretimes,*popt) #fit it to a function that doesn't have the BB ratio
        # fit *= BB_ratio_green(10**popt[-1]) #for green visual
        # fit *= BB_ratio_red(10**popt[-1]) #for red visual

        self.popt,self.pcov = popt,pcov
        self.names = names

        if one_plot:
            plt.figure(figsize=(8,6))
            plt.errorbar(self.time[greenmask],self.flux[greenmask],self.err[greenmask],fmt='.',c='green',label='ZTF_g data',zorder=-1,alpha=0.7)
            plt.errorbar(self.time[~greenmask],self.flux[~greenmask],self.err[~greenmask],fmt='.',c='red',label='ZTF_r data',zorder=-1,alpha=0.7)
            # plt.scatter(time,flux,s=5,c='black',label='Data')
            plt.scatter(self.peak_position_guess,self.peak_guess,s=50,marker='x',c='blue',zorder=10,label=r'Least $\chi^2$')
            # plt.scatter(popt[1],10**popt[0],s=50,marker='x',c='pink',zorder=10,label=r'Fitted peak',alpha=0.9)

            plt.plot(moretimes,fit,label='Piecewise fit',zorder=9,c='black')
            # plt.vlines(popt[1],0,150,label='Found peak',colors='red')
            plt.xlabel(f"Time (mjd), ref jd {self.time_zeropoint}")
            plt.ylabel("Flux (uJy)")
            plt.title("ZTF18abjjkeo")
            plt.legend()  
            plt.show()
        if subplot:
            pass





In [53]:
test = ZTF_forced_phot(test_ztf)
test.fit()



IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed